In [1]:
using ForwardDiff
using Plots

In [2]:
# Dubins car parallel park
n = 3 # number of states
m = 2 # number of controls

# initial and goal states
x0 = [0.;0.;0.]
xf = [0.;1.;0.]

# costs
Q = 0.001*eye(n)
Qf = 1000.0*eye(n)
R = 0.001*eye(m)

dt = 0.01

# RK4 integration
function fc(x,u)
    return [u[1]*cos(x[3]); u[1]*sin(x[3]); u[2]]
end

function rk4(f::Function,dt::Float64)
    # Runge-Kutta 4
    k1(x,u) = dt*f(x,u)
    k2(x,u) = dt*f(x + k1(x,u)/2.,u)
    k3(x,u) = dt*f(x + k2(x,u)/2.,u)
    k4(x,u) = dt*f(x + k3(x,u), u)
    fd(x,u) = x + (k1(x,u) + 2.*k2(x,u) + 2.*k3(x,u) + k4(x,u))/6.
end

function Jacobian(f::Function,x::Array{Float64,1},u::Array{Float64,1})
    f1 = a -> f(a,u)
    f2 = b -> f(x,b)
    fx = ForwardDiff.jacobian(f1,x)
    fu = ForwardDiff.jacobian(f2,u)
    return fx, fu
end

fd = rk4(fc, dt)
F(x,u) = Jacobian(fd,x,u)

X = [1.;1.;1.]
U = [1.;1.]

2-element Array{Float64,1}:
 1.0
 1.0

In [3]:
function gson(X)
    #Gram-Schmidt orthonormalization which produces the same result as [Q,R]=qr(X,0)
    #Written by Mo Chen (sth4nth@gmail.com).
    d, n = size(X)
    m = min(d,n)
    R = zeros(m,n)
    Q = zeros(d,m)
    for i = 1:m
        R[1:i-1,i] = Q[:,1:i-1]'*X[:,i]
        v = X[:,i]-Q[:,1:i-1]*R[1:i-1,i]
        R[i,i] = norm(v)
        Q[:,i] = v/R[i,i]
    end
    R[:,m+1:n] = Q'*X[:,m+1:n]
        
    return Q, R
end

gson (generic function with 1 method)

In [4]:
# Standard backward pass
@time begin 
    S = Qf
    s = Qf*(X - xf)

    lx = Q*(X - xf)
    lu = R*(U)
    lxx = Q
    luu = R
    fx, fu = F(X,U)
    Qx = lx + fx'*s
    Qu = lu + fu'*s
    Qxx = lxx + fx'*S*fx
    Quu = luu + fu'*S*fu
    Qux = fu'*S*fx
#     K = Quu\Qux
#     d = Quu\Qu
#     s_ = (Qx' - Qu'*K + d'*Quu*K - d'*Qux)'
    S_ = Qxx + K'*Quu*K - K'*Qux - Qux'*K
end
println("S=\n$S\nS_=\n$S_")

LoadError: [91mUndefVarError: K not defined[39m

In [6]:
## Square-root backward pass
@time begin 
    S = Qf
    s = Qf*(X - xf)
    L = chol(S)
    Lq = chol(Q)
    Lr = chol(R)

    A, B = F(X,U) # dynamics Jacobians

    G = L'*B*inv(Lr')
    d1, W1 = eig(G*G')
    D1 = diagm(d1)
    P = [A'*L*W1*((eye(size(D1,1)) + D1)^(-1/2))*W1' Lq]

    W1, V = gson(P)
    d2, W2 = eig(V*V')
    D2 = diagm(d2)
    L_ = W1*W2*D2^(1/2)
end
# check result
# isapprox.(S_,L_*L_')

  0.000436 seconds (412 allocations: 39.203 KiB)


3×3 Array{Float64,2}:
 -0.364655  1.64711    26.6944  
 -0.603386  2.58695   -16.9524  
  3.0667    0.704847   -0.161274

In [ ]:
P = 10.*eye(3)
M = ones(2,3)
Fm = 1.*eye(3)
Q = 0.1*eye(3)
R = 0.05*eye(2)
Ps = chol(P)
Qs = chol(Q)
Rs = chol(R)

A = [Fm*Ps Qs]

L, V = gson(A)
D = V*V'

Ps_ = L*D^(1/2)
Ps_*Ps_'

Bt = inv(Rs)*M*Ps

W, V = gson(Bt'*Bt)

#Ps__ = Ps_*W*((eye(3) + V*V')^(-1/2))*W'

Ps__ = Ps_*inv(eye(3) + Bt'*Bt)*Ps_'
Ps__*Ps__'

In [ ]:
Pnew = Fm*P*Fm' + Q

In [ ]:
P_new = Pnew - Pnew*M'*inv(M*Pnew*M' + R)*M*Pnew

In [ ]:
vals, vectors = eig(eye(3))
diagm(vals)
vectors*vectors'

In [ ]:
B = [1. 0. 0.; 0. 1. 0.]
B'*B

V,D = eig(B'*B)

In [1]:
n = 3
m = 2
P = 10.*eye(n)
M = 1.*ones(m,n)
Q = 3.*eye(m)
Ps = chol(P)

P_ = P - P*M'*inv(M*P*M' + Q)*M*P

3×3 Array{Float64,2}:
  6.8254  -3.1746  -3.1746
 -3.1746   6.8254  -3.1746
 -3.1746  -3.1746   6.8254

In [2]:
Bt = inv(chol(Q))*M*Ps

P_2 = Ps*inv(eye(n) + Bt'*Bt)*Ps'
P_2

3×3 Array{Float64,2}:
  6.8254  -3.1746  -3.1746
 -3.1746   6.8254  -3.1746
 -3.1746  -3.1746   6.8254

In [13]:
eigs, vec = eig(Bt'*Bt)

eigs2, vec2 = eig([Bt' zeros(3,1)])

vec2

3×3 Array{Float64,2}:
 0.0  0.57735   0.707107
 0.0  0.57735  -0.707107
 1.0  0.57735   0.0     